In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
import matplotlib.pyplot as plt
from pandas_datareader.data import Options
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import sched
import time
from collections import OrderedDict
%matplotlib inline
import pandas_datareader.data as web
from helpers import *

In [5]:
start_time = time.time()

dte_thresh = 2
money_thresh = 0.02
tick = '^SPX'
deltas_thresh = [-0.5,-0.01]

spx_options = all_options(tick)

spx_filtered_puts = spx_options[(spx_options['DTE'] <= dte_thresh) &
                                (spx_options['Type'] == 'put') &
                                (spx_options['Delta'] >= deltas_thresh[0]) &
                                (spx_options['Delta'] <= deltas_thresh[1])].reset_index()[spx_options.columns].sort_values('Strike',
                                                                                                                ascending = False)

spreads_lst = []
for idx, row in spx_filtered_puts.iterrows():
    if idx == 0:
        break
    
    temp_chains = spx_filtered_puts[spx_filtered_puts['Strike'] < row.Strike][['Strike','Bid','Ask',
                                                                               'Delta','Gamma','Theta','Vega']]
    temp_chains['DTE'] = row.DTE
    temp_chains['Short Strike'] = row.Strike
    temp_chains['Short Bid'] = row.Bid
    temp_chains['Short Ask'] = row.Ask
    
    temp_chains['Premium'] = (temp_chains['Short Bid'] - temp_chains['Ask'])*100
    temp_chains['Total Risk'] = (temp_chains['Strike'] - temp_chains['Short Strike'])*100 + temp_chains['Premium']
    temp_chains['Delta'] = temp_chains['Delta'] - row.Delta
    temp_chains['Gamma'] = temp_chains['Gamma'] - row.Gamma
    temp_chains['Theta'] = temp_chains['Theta'] - row.Theta
    temp_chains['Vega'] = temp_chains['Vega'] - row.Vega
    
    temp_chains = temp_chains[['DTE','Short Strike','Strike','Premium','Delta','Gamma','Theta','Vega','Total Risk']]
    spreads_lst.append(temp_chains)
    
pspreads_df = pd.concat(spreads_lst, axis = 0).reset_index()[temp_chains.columns]
pspreads_df['RRRatio'] = pspreads_df.Premium/(-pspreads_df['Total Risk'])

pspreads_df = pspreads_df.sort_values('RRRatio',ascending = False)

print("--- %s seconds ---" % (time.time() - start_time))



--- 46.79334235191345 seconds ---
